In [1]:
import requests
import pandas as pd

In [21]:
access_token = ''

In [22]:
headers = {'Authorization': 'Bearer {}'.format(access_token)}
user_url = 'https://api.spotify.com/v1/me'

r = requests.get(user_url, headers=headers)

In [23]:
r.text

'{\n  "country" : "US",\n  "display_name" : "guidopetri",\n  "email" : "guidopetri@gmail.com",\n  "explicit_content" : {\n    "filter_enabled" : false,\n    "filter_locked" : false\n  },\n  "external_urls" : {\n    "spotify" : "https://open.spotify.com/user/guidopetri"\n  },\n  "followers" : {\n    "href" : null,\n    "total" : 8\n  },\n  "href" : "https://api.spotify.com/v1/users/guidopetri",\n  "id" : "guidopetri",\n  "images" : [ ],\n  "product" : "premium",\n  "type" : "user",\n  "uri" : "spotify:user:guidopetri"\n}'

In [24]:
user_data = r.json()
user_data.keys()

dict_keys(['country', 'display_name', 'email', 'explicit_content', 'external_urls', 'followers', 'href', 'id', 'images', 'product', 'type', 'uri'])

In [25]:
playlists_url = f'https://api.spotify.com/v1/users/{user_data["id"]}/playlists'
params = {'limit': 50}

playlists = []

while playlists_url is not None:
    r = requests.get(playlists_url, params=params, headers=headers)
    
    data = r.json()
    
    playlists.extend(data['items'])
    playlists_url = data['next']

In [26]:
names = [x['name'] for x in playlists]
start = names.index('Deal with later') + 1
end = names.index('Shazam')
names = names[start:end]

In [27]:
filtered_playlists = [playlist for playlist in playlists if playlist['name'] in names]

In [28]:
filtered_playlists[0]['tracks']

{'href': 'https://api.spotify.com/v1/playlists/7IRnBmuIaf3GfPfjLm8UDN/tracks',
 'total': 20}

In [29]:
params = {'limit': 100, 'fields': 'items(track(id)),next'}

playlist_tracks = {}

for playlist in filtered_playlists:
    tracks_url = playlist['tracks']['href']
    tracks = []
    
    while tracks_url is not None:
        r = requests.get(tracks_url, headers=headers, params=params)
        
        data = r.json()
        
        tracks.extend([x['track']['id'] for x in data['items']])
        
        tracks_url = data['next']
    
    playlist_tracks[playlist['name']] = tracks

In [40]:
x = []
x.append([y for y in zip(cycle([playlist['name']]), tracks)])
x

[[('Videogame', '3kK24sf6YfIvbJfGC48vN7'),
  ('Videogame', '0prSDXsNF1lSh3rsVJS91W'),
  ('Videogame', '6KuqKhcEn0vlT8QegxbM7m'),
  ('Videogame', '6ksdQIV6qpxdanDbmSIvtx'),
  ('Videogame', '108dAAiaSorOEJMMOPVlzK'),
  ('Videogame', '6LlIPRpFJMDcw8b4b7eSlq'),
  ('Videogame', '5sYgsJJpbyfKCm8s4cRrWE'),
  ('Videogame', '6rCQkxKkNSLksQJZGI2A8I'),
  ('Videogame', '7awSiJ4aLoUb1AUDl32HFy'),
  ('Videogame', '67gAKzJfIr8xzVEcS3dDqr'),
  ('Videogame', '6QlJ9q2zyN1ABQnDjwEyP3'),
  ('Videogame', '7Aj6EzwA0xLNN5XnUI7kTD'),
  ('Videogame', '3QVKVC7x6ka08um8fz7SEx'),
  ('Videogame', '50Iox7zPgdsnn7teyvmCgh'),
  ('Videogame', '2Onfq0K0IsF2rxgufvYJLV'),
  ('Videogame', '36oCvQSDpJzDNUMRvYyDRe'),
  ('Videogame', '7x9XYTscRLGrwRkDLlFRe8'),
  ('Videogame', '6a0ggIME71sZdFlnY3ARy7'),
  ('Videogame', '1KIlKiOeAjKXDwdocHuHUz'),
  ('Videogame', '7hXIA7fZHRXI2PjgkUf7Cx'),
  ('Videogame', '1WL7wxZB266w7SeO6aLZ9Q'),
  ('Videogame', '2NsaQhLEmCcziiS2HB1C8t'),
  ('Videogame', '7zx0r1pcEiX92UwJ3MuNbV'),
  ('Videoga

In [30]:
playlist_tracks.keys()

dict_keys(['Breaks', 'Brostep', 'Country', 'Dance', 'Deep House', 'Disco', 'Downtempo', 'Drum n Bass', 'Drumstep', 'Dubstep', 'Dutch House', 'Electro', 'Electro House', 'Electro Swing', 'Funky House', 'G House', 'Garage', 'Glitch Hop', 'Grime', 'Hard Trance', 'Hardstyle', 'Heavy Metal', 'House', 'Instrumental Hip-Hop', 'Old Skool', 'Pop', 'Pop House', 'Progressive House', 'Psytrance', 'Rap', 'Reggae', 'Rock', 'Tech House', 'Trance', 'Trancestep', 'Trap', 'Trop House', 'Videogame'])

In [31]:
from itertools import cycle

In [20]:
playlists_df = pd.DataFrame((k, v) for k in playlist_tracks.keys() for v in playlist_tracks[k])
#playlists_df.set_index(1, inplace=True)
playlists_df.columns = ['genre', 'track_id']

In [24]:
playlists_df.to_csv('playlists-tracks.csv')